## Example usage for climates outside of CA

In [1]:
import unittest
import os

# please restart the kernel if rerunning this for the paths to align
os.chdir('../')

import numpy as np
import pandas as pd

from mswh.comm.label_map import SwhLabels
from mswh.system.components import Converter, Storage
from mswh.system.source_and_sink import SourceAndSink
from mswh.system.models import System
from mswh.system.source_and_sink import SourceAndSink

from mswh.tools.unit_converters import UnitConv, Utility
from mswh.comm.sql import Sql

from mswh.tools.plots import Plot
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
sns.set(rc={'figure.figsize':(16,12)})
sns.set_style("whitegrid")

import datetime

import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

# label maps
c = SwhLabels().set_hous_labels()
s = SwhLabels().set_prod_labels()
r = SwhLabels().set_res_labels()

# pd.options.display.float_format = '{:,.3f}'.format


# Multiscale Solar Water Heating System Simulation Tool 

## Contents


* **Project input (minimum user requirment)**
* **Weather (CA climate zones available in the database)** 
* **Loads (example CA hourly loads available in the database)**
* **Performance paramters (defaults provided, user edits optional, such as providing insulation thickness for tanks)**
* **Component sizing (sizing rules provided, user edits optional, such as providing the actual length of the piping)**
* **System simulation examples with plots and summaries for all system configurations**

### System configurations
* Basecase gas tank water heater (WH)
* Active solar thermal system with a new tankless gas WH backup
* Active solar thermal system with a retrofit tankless gas WH backup


# Setup 
## Import modules 

## Project inputs
* The user in the default case only needs to specify household occupancy and a climate in CA

In [2]:
# Add a value to the list for each additional household in a community scale project

# Community scale household occupancies for 4 households
occ_com = [3, 3, 3, 3]

# Individual scale household occupancy
occ_ind = [3]

# indicate whether any occupants are at home during the day
at_home_com = ['n', 'n', 'n', 'n']
at_home_ind = ['n']

# Example climate zone : Banja Luka, Bosnia and Hercegowina
bl_climate = '00'

## Load in input database (DB)

In [3]:
weather_db_path = os.path.join(os.getcwd(), 'mswh/comm/mswh_system_input.db')

db = Sql(weather_db_path)

try:
    # read table names for all tables in a
    # {table name : sheet name} form
    inputs = db.tables2dict(close = True)
except:
    msg = 'Failed to read input tables from {}.'
    log.error(msg.format(inpath))

## Generate weather data
* Includes calculation of the incident irradiation with the default tilt and azimuth

In [4]:
source_and_sink = SourceAndSink(\
    input_dfs = inputs)

# Using TMY data for Banja Luka
weather_bl = source_and_sink.irradiation_and_water_main('00',
    weather_data_source = 'tmy3',
    method='isotropic diffuse')

INFO:mswh.system.source_and_sink:The user did not provide a numpy random state object. Initiating one with a provided or default seed value = 123.


In [5]:
weather_bl.head(30)

,date (mm/dd/yyyy),time (hh:mm),extraterrestrial_horizontal_radiation_Wm2,extraterrestrial_normal_radiation_Wm2,global_horizontal_radiation_Wm2,ghi source,ghi uncert (%),direct_normal_radiation_Wm2,dni source,dni uncert (%),...,day_number_of_year,season,beam_horizontal_radiation_Wm2,global_tilt_radiation_Wm2,water_main_t_F,water_main_t_C,dry_bulb_C,wet_bulb_C,Tilt,Azimuth
0,01/01/05,01:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,0.8,0.259183,17.217,0.0
1,01/01/05,02:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-1.8,-2.178397,17.217,0.0
2,01/01/05,03:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-1.9,-2.278473,17.217,0.0
3,01/01/05,04:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-2.1,-2.478625,17.217,0.0
4,01/01/05,05:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-2.2,-2.502155,17.217,0.0
5,01/01/05,06:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-2.4,-2.702735,17.217,0.0
6,01/01/05,07:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-2.5,-2.803025,17.217,0.0
7,01/01/05,08:00,0,0,0,0,0,0,0,0,...,1,winter,0,0.000000,51.6,10.888889,-2.7,-3.003606,17.217,0.0
8,01/01/05,09:00,189,1298,24,0,0,0,0,0,...,1,winter,0,23.462287,51.6,10.888889,-1.9,-2.201285,17.217,0.0
9,01/01/05,10:00,355,1415,61,0,0,0,0,0,...,1,winter,0,59.633313,51.6,10.888889,-1.2,-1.577944,17.217,0.0


## Select hourly hot water draw profiles from DB

In [6]:
# Returns hourly load profiles in m3/h and the peak load in gal/h for each household in the project

# as water demand in California is low, a quick way to adjust it is to increase the number of occupants

rs = np.random.RandomState(123)

# peakload_com output shows the consumer ID, load ID and the peak hourly end-use load
# loads_com holds the loads itself, with the associated consumer ID and occupancy

loads_com, peakload_com = \
    SourceAndSink._make_example_loading_inputs(
    inputs, c, rs,
    occupancy = list(occ_com + np.ones(len(occ_com)).astype(int)), at_home = at_home_com)

loads_indiv, peakload_indiv = \
    SourceAndSink._make_example_loading_inputs(
    inputs, c, rs,
    occupancy = [occ_ind[0] + 1], at_home = at_home_ind)

In [7]:
[occ_ind[0] + 1]

[4]

In [8]:
peakload_indiv

,Consumer ID,Load ID,Peak End-Use Load [gal]
0,1,122,44.25


## Set performance parameters

* Active solar thermal system

In [9]:
# Community scale system (includes distribution pump parameters)

sol_the_sys_params_com = pd.DataFrame(data = \
            [[s['the_sto'], s['f_upper_vol'], .5], \
             [s['the_sto'], s['ins_thi'], .085], \
             [s['the_sto'], s['spec_hea_con'], .04], \
             [s['the_sto'], s['t_tap_set'], 322.04], \
             [s['the_sto'], s['h_vs_r'], 6.], \
             [s['the_sto'], s['dt_appr'], 2.], \
             [s['the_sto'], s['t_max_tank'], 344.15], \
             [s['the_sto'], s['eta_coil'], .84], \
             [s['the_sto'], s['circ'], 0.], \
             [s['sol_col'], s['interc_hwb'], .753], \
             [s['sol_col'], s['slope_hwb'], -4.025],\
             [s['sol_pump'], s['eta_sol_pump'], .85], \
             [s['piping'], s['pipe_spec_hea_con'], .0175], \
             [s['piping'], s['pipe_ins_thick'], .008], \
             [s['piping'], s['flow_factor'], .8], \
             [s['piping'], s['dia_len_exp'], 0.43082708345352605], \
             [s['piping'], s['dia_len_sca'], 0.007911283766743384],\
             [s['piping'], s['discr_diam_m'], \
               '[0.0127, 0.01905, 0.0254, '\
               '0.03175, 0.0381, 0.0508, 0.0635, 0.0762, 0.1016]'],\
             [s['piping'], s['circ'], False], \
             [s['piping'], s['long_br_len_fr'], 1.], \
             [s['dist_pump'], s['eta_dist_pump'], .85]], \
            columns = \
             [s['comp'], s['param'], s['param_value']])

sol_the_sys_params_com

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


In [10]:
# Individual scale system, assuming a distribution pump is not needed similarly to a basecase gas tank WH

last_row_for_indiv = sol_the_sys_params_com.shape[0] - 1 - \
    sol_the_sys_params_com.loc[\
    sol_the_sys_params_com[s['comp']] == s['dist_pump'], :].\
    shape[0]

sol_the_sys_params_indiv = sol_the_sys_params_com.loc[:last_row_for_indiv,:]
sol_the_sys_params_indiv

sol_the_sys_params_indiv

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


* Basecase and backup gas tank water heater (WH)

In [11]:
gas_tank_wh_params = pd.DataFrame(data = \
    [[s['gas_tank'], s['tank_re'], .78, '-'], \
     [s['gas_tank'], s['ins_thi'], .03, 'm'], \
     [s['gas_tank'], s['spec_hea_con'], .081, 'W/mK'], \
     [s['gas_tank'], s['t_tap_set'], 322.04, 'K']], \
    columns = \
     [s['comp'], s['param'], s['param_value'], s['param_unit']])

gas_tank_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas tank WH,tank recovery efficiency,0.780,-
1,gas tank WH,insulation thickness,0.030,m
2,gas tank WH,specific heat conductivity,0.081,W/mK
3,gas tank WH,tap temperature setpoint,322.040,K


* Gas tankless WH

In [12]:
gas_tankless_wh_params = pd.DataFrame(data = [[s['gas_burn'], s['comb_eff'], .85, '-']], \
    columns = [s['comp'], s['param'], s['param_value'], s['param_unit']])
gas_tankless_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas burner,combustion efficiency,0.85,-


* All performance parameters

In [13]:
# this input table provides all available component performance parameters
inputs['component_performance']

,Component ID,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,1,interc hwb,0.753,-
1,1,slope hwb,-4.025,W/m2K
2,1,interc cd,0.75,-
3,1,a1 cd,-3.688,W/m2K
4,1,a2 cd,-0.0055,W/m2K2
5,4,PV efficiency,0.16,-
6,4,fraction of active PV area,1,-
7,4,reference irradiation,1000,W/m2
8,5,tank recovery efficiency,0.78,-
9,5,tap temperature setpoint,322.04,K


In [14]:
inputs['sys_3_components']

,Component ID,Component,Component Technology,Component Size Unit
0,1,solar collector,flat plate,m2
1,2,solar collector,tubular,m2
2,3,pv,monochrystalline,W
3,4,pv,polychrystalline,W
4,5,gas tank WH,conventional gas tank water heater,m3
5,6,electric resistance tank WH,conventional electric resistance tank water he...,m3
6,7,thermal storage tank,thermal storage tank with an in-tank coil,m3
7,8,gas burner,gas burner,W
8,9,electric resistance heater,in-tank or instantaneous,W
9,10,heat pump,in-tank,W


In [15]:
to_doc = inputs['component_performance'].merge(inputs['sys_3_components'], on = "Component ID", how = "left")

## Component sizing

Note that backup is sized to full load.

* Gas tank WH: DOE sizing rule based on peak hourly demand +-2 gal

In [16]:
# Basecase gas tank WH

# in m3
gas_tank_size_indiv = UnitConv(peakload_indiv.loc[0, c['max_load']]).m3_gal(unit_in = 'gal')
gas_tank_size_indiv

# Individual household, backup
# same as the basecase sizing

0.167504481

In [17]:
# Community scale, backup

# get peak loads for each household
peakload_com['gas_tank_size_com'] = peakload_com[\
    c['max_load']].apply(lambda x: UnitConv(x).m3_gal(unit_in = 'gal'))
peakload_com.index = peakload_com[c['id']]

peakload_com

,Consumer ID,Load ID,Peak End-Use Load [gal],gas_tank_size_com
Consumer ID,,,,
1,1,9,43.70,0.165423
2,2,58,44.90,0.169965
3,3,7,41.65,0.157662
4,4,29,40.70,0.154066


* Active solar thermal system: CSI Handbook sizing rules

In [18]:
# CSI demand estimate
def demand_estimate(occ):
    if occ == 1:
        return 20.
    if occ == 2:
        return 35.
    else:
        return 35.+10.*(occ-2.)
    
col_area_scaler = 1.2 #(CSI sizing rule: upper limit 1.25)
tank_vol_scaler = 1.3 #(CSI sizing rule: lower limit 1.25)

# Individual household (without a distribution pump)

# get demand estimate
demand_estimate_ind = demand_estimate(occ_ind[0])

# CSI sizing rules
col_area_ind_sqft = demand_estimate_ind * col_area_scaler
tank_vol_ind_gal = col_area_ind_sqft * tank_vol_scaler

col_area_ind = UnitConv(col_area_ind_sqft).sqft_m2()
tank_vol_ind = UnitConv(tank_vol_ind_gal).m3_gal(unit_in = 'gal')

# Community scale (with a distribution pump)

# get demand estimate
demand_estimate_com = 0.
for inx, row in loads_com.iterrows():
    demand_estimate_com += demand_estimate(row[c['occ']])

col_area_com_sqft = demand_estimate_com * col_area_scaler
tank_vol_com_gal = col_area_com_sqft * tank_vol_scaler

col_area_com = UnitConv(col_area_com_sqft).sqft_m2()
tank_vol_com = UnitConv(tank_vol_com_gal).m3_gal(unit_in = 'gal')

* Gas tankless WH: Based on the WH efficiency standards analysis and input powers collected online

In [19]:
def gas_tankles_size_W(occupancy):
    size = 24875.*occupancy**0.5175
    return size

* Solar pump

In [20]:
# solar pump
solar_pump_w_indiv = 7.5101 * sum(occ_ind) ** 0.5322
solar_pump_w_com = 7.5101 * sum(occ_com) ** 0.5322

### Distribution system

* Piping

In [21]:
# If the length of community piping installation is unknown, use this estimate

# individual 
pipe_m_indiv = 0 # this can be set to a real value if it is known. It is set to 0 here since the part of piping inside of a 
# household is assumed to have negligible thermal losses

# community scale

pipe_m_per_hhld = 3.048
# household type distribution pipe length scalers for communities with attached and detached single family houses
multifamily = 1.
attached_k = 3.
detached_k = 6.

diameter_scaler=0.00791128
diameter_exponent=0.430827

# The average diameter of the distribution piping is a function of distribution pipe length and it gets autosized 
# within the distribution model, where the first larger value than that theoretically obtained gets selected
# from this list: [0.0127, 0.01905, 0.0254, 0.03175, 0.0381, 0.0508, 0.0635, 0.0762, 0.1016], in m

# The formula is:

# d_m = diameter_scaler * (pipe_m_per_hhld * number_of_households)**diameter_exponent
# To change these parameters, edit values in sol_the_sys_params_{com/indiv} dataframes

d_m = diameter_scaler * (pipe_m_per_hhld * len(occ_com))**diameter_exponent

d_m

0.02323577539362201

* Distribution pump

In [22]:
dist_pump_w = 10.4376 * len(occ_com) ** 0.9277

## Compile component sizes for system simulation

* Basecase gas tank WH

In [23]:
# Individual household

gas_tank_wh_size = pd.DataFrame(
    data = [[s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [s['comp'], s['cap'], s['cap_unit']])

gas_tank_wh_size

,Component,Component Size,Component Size Unit
0,gas tank WH,0.167504,m3


* Solar thermal system, individual household

In [24]:
# Main system

sys_sizes_indiv = pd.DataFrame(data = \
    [[s['sol_col'], col_area_ind, 'm2'], \
     [s['the_sto'], tank_vol_ind, 'm3'], \
     [s['sol_pump'], solar_pump_w_indiv, 'W'], \
     [s['piping'], pipe_m_indiv, 'm']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

sys_sizes_indiv

,Component,Component Size,Component Size Unit
0,solar collector,5.016764,m2
1,thermal storage tank,0.265736,m3
2,solar pump,13.476268,W
3,piping,0.000000,m


In [25]:
# Retrofit: Backup gas tank WH

retrofit_bckp_size_indiv = pd.DataFrame(data = \
            [[1, s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])
demand_estimate_com

220.0

In [26]:
# New: Backup gas tankless WH

new_bckp_size_ind = pd.DataFrame(data = [[1, s['gas_burn'], gas_tankles_size_W(occ_ind[0]), 'W']], \
            columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])

new_bckp_size_ind

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas burner,43921.113177,W


* Solar thermal system, community

In [27]:
# Main system components

sys_sizes_com = pd.DataFrame(data = \
            [[s['sol_col'], col_area_com, 'm2'], \
             [s['the_sto'], tank_vol_com, 'm3'], \
             [s['sol_pump'], solar_pump_w_com, 'W'], \
             [s['dist_pump'], dist_pump_w, 'W'], \
             [s['piping'], multifamily * pipe_m_per_hhld * len(occ_com),'m']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

In [28]:
# Retrofit: Backup gas tank WHs

retrofit_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']],
                                      index = peakload_com.index)

for i in peakload_com.index:
    retrofit_bckp_size_com.loc[i, c['id']] = i
    retrofit_bckp_size_com.loc[i, s['comp']] = s['gas_tank']
    retrofit_bckp_size_com.loc[i, s['cap']] = peakload_com.loc[i, 'gas_tank_size_com'].round(3)

retrofit_bckp_size_com.index = range(retrofit_bckp_size_com.shape[0])
retrofit_bckp_size_com[s['cap_unit']] = 'm3'
retrofit_bckp_size_com

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas tank WH,0.165,m3
1,2,gas tank WH,0.17,m3
2,3,gas tank WH,0.158,m3
3,4,gas tank WH,0.154,m3


In [29]:
# New: Backup gas tankless WHs

new_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']], index = peakload_com.index)

for i in peakload_com.index:
    new_bckp_size_com.loc[i, c['id']] = i
    new_bckp_size_com.loc[i, s['comp']] = s['gas_burn']
    new_bckp_size_com.loc[i, s['cap']] = gas_tankles_size_W(occ_com[i - 1])

new_bckp_size_com.index = range(new_bckp_size_com.shape[0])

* Validation sizing

Sizes based on several observed rated systems from OG-300 with an 80 gal tank and gas tankless backup
(https://secure.solar-rating.org/Certification/Ratings/RatingsReport.aspx?device=6926&units=METRICS)


In [30]:
# Solar thermal with tankless backup
sys_sizes_val = pd.DataFrame(data = \
            [[s['sol_col'], 3.9], \
             [s['the_sto'], UnitConv(80.).m3_gal(unit_in ='gal')], \
             [s['sol_pump'], solar_pump_w_indiv], \
             [s['piping'], 0.]], \
            columns = \
             [s['comp'], s['cap']])

sys_sizes_val

,Component,Component Size
0,solar collector,3.900000
1,thermal storage tank,0.302833
2,solar pump,13.476268
3,piping,0.000000


In [31]:
# visualisation setup
pd.options.display.float_format = '{:,.3f}'.format
w = 650
h = 450
f = 16
m_l = 60.
m_b = 60.

# hours of the year plotted time, a week in July starting on Monday
first_hour = 29*7*24 + 1
last_hour = first_hour + 7*24
first_hour

4873

# System Simulation
## Basecase gas tank WH

In [32]:
# cold climate

conv_gas_tank_wh_cold = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_bl, \
            loads = loads_indiv)

cons_total_cold, proj_total_cold, ts_proj_cold = conv_gas_tank_wh_cold.conventional_gas_tank()

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [33]:
proj_total_cold/1000

Consumer ID                                    0.001
Occupancy                                      0.004
Energy Use - Gas                           4,607.702
Energy Use - Gas, Summer                   1,884.095
Energy Use - Gas, Winter                   2,723.607
Energy Use - Electricity                       0.000
Energy Use - Electricity, Summer               0.000
Energy Use - Electricity, Winter               0.000
Solar Fraction                                 0.000
Backup Heat Delivered                            NaN
Unmet Heat                                     0.000
Energy Use - Gas, w/o Dist Loss            4,607.702
Energy Use - Gas, Summer, w/o Dist Loss    1,884.095
Energy Use - Gas, Winter, w/o Dist Loss    2,723.607
Total Heat Delivered                       2,562.274
Net Heat Demand                            2,562.274
Dumped Heat                                    0.000
Distribution Heat Loss                         0.000
Distribution Heat Loss at Backup              

In [34]:
data_bc = pd.DataFrame(data = \
        [ts_proj_cold[r['gas_use']]]).transpose()

fig = Plot(data_headers = ['Gas Use, BL Climate Zone'], \
                 outpath = '', save_image = False, title = '', label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.01, legend_y = 1.).\
            series(data_bc[first_hour: last_hour], outfile = '', index_in_a_column = None, modes = 'lines')

## Individual new solar thermal system

In [35]:
# in a cold climate
sol_wh_indiv_new_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = new_bckp_size_ind,
                          weather = weather_bl,
                          loads = loads_indiv)

cons_total_indiv_new_cold, proj_total_indiv_new_cold,\
[proj_total_dict_indiv_new_cold, sol_fra_indiv_new_cold, pump_el_use_indiv_new_cold,\
 pump_op_hour_indiv_new_cold, ts_res_indiv_new_cold, backup_ts_cons_indiv_new_cold, rel_err_indiv_new_cold] = \
    sol_wh_indiv_new_cold.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.


In [36]:
print('month, day, hour:\nData starts at', ts_res_indiv_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Individual SWH New' + ', Climate Zone ' + str(int(cold_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.4).\
            series(ts_res_indiv_new_cold.loc[first_hour : last_hour,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [37]:
ts_res_indiv_new_cold.keys()

Index(['Net Heat Demand', 'Heat Demand With Dist. Loss',
       'Demand Balancing Error (Heat Rate)', 'Solar Heat Delivered To Tank',
       'Heat Loss - Lower Tank Volume', 'Heat Loss - Upper Tank Volume',
       'Tank Heat Delivered', 'Tank Unmet Heat', 'Dumped Heat',
       'Tank Overcool (Heat Rate)', 'Temperature - Upper Tank Volume',
       'Temperature - Lower Tank Volume', 'Temperature - Tank Coil Out',
       'Temperature - Hot Water Set', 'Temperature Drop in Distribution Pipes',
       'Distribution Heat Loss', 'Distribution Heat Loss at Backup',
       'Distribution Heat Loss at Backup, Summer',
       'Distribution Heat Loss at Backup, Winter', 'Backup Heat Delivered',
       'Energy Use - Gas', 'Energy Use - Gas, Summer',
       'Energy Use - Gas, Winter', 'Energy Use - Gas, w/o Dist Loss',
       'Energy Use - Gas, Summer, w/o Dist Loss',
       'Energy Use - Gas, Winter, w/o Dist Loss', 'Unmet Heat',
       'Total Heat Delivered', 'Temperature - Ambient',
       'Temper

In [38]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Individual SWH New, Climate Zone ' + str(int(bl_climate)), 
                 label_h = 'Hour [h]', label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.01, legend_y = 1.15).\
            series(ts_res_indiv_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

## Community new solar thermal system

In [39]:
# in a cold climate
sol_wh_com_new_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = new_bckp_size_com,
                          weather = weather_bl,
                          loads = loads_com)

cons_total_com_new_cold, proj_total_com_new_cold,\
[proj_total_dict_com_new_cold, sol_fra_com_new_cold, pump_el_use_com_new_cold,\
 pump_op_hour_com_new_cold, ts_res_com_new_cold, backup_ts_cons_com_new_cold, rel_err_com_new_cold] = \
    sol_wh_com_new_cold.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.


In [40]:
# first_hour = 4897
# last_hour = 4897 + 5*24

print('month, day, hour:\nData starts at', ts_res_com_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

# title = 'Community SWH New, Climate Zone ' + str(int(cold_cz))

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = '', 
                 label_h = 'Hour [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.65, legend_y = 1.3).\
            series(ts_res_com_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


### Compare new installation systems across climates and scales

In [41]:
# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Per Household, Community, New, BL Climate Zone ']
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual, New, BL Climate Zone ']
compare_cold = df.append(df1).transpose()

compare_cold # this is bl climate

,"Per Household, Community, New, BL Climate Zone","Individual, New, BL Climate Zone"
Consumer ID,2.500,1.000
Occupancy,4.000,4.000
Energy Use - Gas,"760,340.312","822,081.741"
"Energy Use - Gas, Summer","6,080.206","16,409.765"
"Energy Use - Gas, Winter","754,260.106","805,671.976"
Energy Use - Electricity,"30,814.320","45,993.710"
"Energy Use - Electricity, Summer","14,723.080","22,545.000"
"Energy Use - Electricity, Winter","16,091.240","23,448.710"
Solar Fraction,0.758,0.730
Backup Heat Delivered,"646,289.265","698,769.480"


In [42]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare_cold.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]/1000).round(0)#.to_clipboard()

,"Per Household, Community, New, BL Climate Zone","Individual, New, BL Climate Zone"
Energy Use - Gas,760,822
"Energy Use - Gas, Summer",6,16
"Energy Use - Gas, Winter",754,806
"Energy Use - Gas, w/o Dist Loss",753,822
Energy Use - Electricity,31,46
Solar Fraction,0,0
Backup Heat Delivered,646,699
Unmet Heat,0,0
Net Heat Demand,"2,663","2,562"
Total Heat Delivered,"2,684","2,562"


## Individual retrofit solar thermal system

In [43]:
# in a cold climate
sol_wh_indiv_retrofit_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = retrofit_bckp_size_indiv,
                          weather = weather_bl,
                          loads = loads_indiv)

cons_total_indiv_retrofit_cold, proj_total_indiv_retrofit_cold,\
[proj_total_dict_indiv_retrofit_cold, sol_fra_indiv_retrofit_cold, pump_el_use_indiv_retrofit_cold,\
 pump_op_hour_indiv_retrofit_cold, ts_res_indiv_retrofit_cold, backup_ts_cons_indiv_retrofit_cold, rel_err_indiv_retrofit_cold] = \
    sol_wh_indiv_retrofit_cold.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [44]:
print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', in Banja Luka',
                 label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [45]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Individual SWH Retrofit' + ', in Banja Luka', label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

## Community retrofit solar thermal system

In [46]:
# in a cold climate
sol_wh_com_retrofit_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = retrofit_bckp_size_com,
                          weather = weather_bl,
                          loads = loads_com)

cons_total_com_retrofit_cold, proj_total_com_retrofit_cold,\
[proj_total_dict_com_retrofit_cold, sol_fra_com_retrofit_cold, pump_el_use_com_retrofit_cold,\
 pump_op_hour_com_retrofit_cold, ts_res_com_retrofit_cold, backup_ts_cons_com_retrofit_cold, rel_err_com_retrofit_cold] = \
    sol_wh_com_retrofit_cold.solar_thermal(backup = 'retrofit')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [47]:
print('month, day, hour:\nData starts at', ts_res_com_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Community SWH Retrofit in Banja Luka', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.).\
            series(ts_res_com_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


## Compare systems across climates, configurations and scales

### Retrofits

In [48]:
# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, BL Climate ']
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, BL Climate ']
compare_cold = df.append(df1).transpose()

compare_cold

,"Community SWH, Per Household, Retrofit, BL Climate","Individual SWH, Retrofit, BL Climate"
Consumer ID,2,1
Occupancy,4,4
Energy Use - Gas,"2,150,210","2,248,688"
"Energy Use - Gas, Summer","565,002","589,376"
"Energy Use - Gas, Winter","1,585,208","1,659,313"
Energy Use - Electricity,"30,814","45,994"
"Energy Use - Electricity, Summer","14,723","22,545"
"Energy Use - Electricity, Winter","16,091","23,449"
Solar Fraction,1,1
Backup Heat Delivered,"646,029","698,770"


In [49]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare_cold.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]/1000).round(0)#.to_clipboard()

,"Community SWH, Per Household, Retrofit, BL Climate","Individual SWH, Retrofit, BL Climate"
Energy Use - Gas,"2,150","2,249"
"Energy Use - Gas, Summer",565,589
"Energy Use - Gas, Winter","1,585","1,659"
"Energy Use - Gas, w/o Dist Loss","2,143","2,249"
Energy Use - Electricity,31,46
Solar Fraction,0,0
Backup Heat Delivered,646,699
Unmet Heat,0,0
Net Heat Demand,"2,663","2,562"
Total Heat Delivered,"2,683","2,562"


### New and Retrofit

In [50]:
# New installations

# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, New, BL Climate ']
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['Individual SWH, Retrofit, BL Climate ']
compare_cold = df.append(df1).transpose()
compare = compare_cold.copy()


# Retrofits

# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['Community SWH, Per Household, Retrofit, BL Climate ']
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, BL Climate']
compare_cold_retr = df.append(df1).transpose()
compare[compare_cold_retr.columns] = compare_cold_retr

compare

,"Community SWH, Per Household, New, BL Climate","Individual SWH, Retrofit, BL Climate","Community SWH, Per Household, Retrofit, BL Climate","indiv, retrofit, BL Climate"
Consumer ID,2,1,2,1
Occupancy,4,4,4,4
Energy Use - Gas,"760,340","822,082","2,150,210","2,248,688"
"Energy Use - Gas, Summer","6,080","16,410","565,002","589,376"
"Energy Use - Gas, Winter","754,260","805,672","1,585,208","1,659,313"
Energy Use - Electricity,"30,814","45,994","30,814","45,994"
"Energy Use - Electricity, Summer","14,723","22,545","14,723","22,545"
"Energy Use - Electricity, Winter","16,091","23,449","16,091","23,449"
Solar Fraction,1,1,1,1
Backup Heat Delivered,"646,289","698,769","646,029","698,770"


In [51]:
pd.options.display.float_format = '{:,.0f}'.format
# subset of results for the report
(compare.loc[['Energy Use - Gas',
'Energy Use - Gas, Summer',
'Energy Use - Gas, Winter',
'Energy Use - Gas, w/o Dist Loss',
'Energy Use - Electricity',
'Solar Fraction',
'Backup Heat Delivered',
'Unmet Heat',
'Net Heat Demand',
'Total Heat Delivered',
'Tank Unmet Heat',
'Tank Heat Delivered',
'Distribution Heat Loss'
],:]).round(0)#.to_clipboard()

,"Community SWH, Per Household, New, BL Climate","Individual SWH, Retrofit, BL Climate","Community SWH, Per Household, Retrofit, BL Climate","indiv, retrofit, BL Climate"
Energy Use - Gas,"760,340","822,082","2,150,210","2,248,688"
"Energy Use - Gas, Summer","6,080","16,410","565,002","589,376"
"Energy Use - Gas, Winter","754,260","805,672","1,585,208","1,659,313"
"Energy Use - Gas, w/o Dist Loss","752,894","822,082","2,142,528","2,248,688"
Energy Use - Electricity,"30,814","45,994","30,814","45,994"
Solar Fraction,1,1,1,1
Backup Heat Delivered,"646,289","698,769","646,029","698,770"
Unmet Heat,0,0,0,0
Net Heat Demand,"2,662,589","2,562,274","2,662,589","2,562,274"
Total Heat Delivered,"2,683,533","2,562,274","2,683,272","2,562,274"
